<a href="https://colab.research.google.com/github/nahidhasan0004/ML/blob/main/Daily_Forcast_Using_Random_Forest(RAINFALL).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [24]:
import pandas as pd

# Load the uploaded Excel file
file_path = "/content/rainfall dhaka.xlsx"
excel_data = pd.ExcelFile(file_path)

# Display sheet names to understand structure
excel_data.sheet_names


['Sheet1']

In [25]:
# Load data from the first sheet
df = excel_data.parse('Sheet1')

# Display the first few rows of the dataset
df.head()


,Station_Name,Station_ID,Year,Month,D_01,D_02,D_03,D_04,D_05,D_06,...,D_23,D_24,D_25,D_26,D_27,D_28,D_29,D_30,D_31,Unnamed: 35
0,Dhaka,41923,1981,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0.0,6,9,NaN
1,Dhaka,41923,1981,2,2,0,0,0,0,0,...,0,0,0,0,0,0,NaN,NaN,NaN,NaN
2,Dhaka,41923,1981,3,0,0,0,0,0,0,...,10,0,13,0,8,0,2.0,0,41,NaN
3,Dhaka,41923,1981,4,2,0,0,4,0,0,...,0,0,0,0,0,0,3.0,0,NaN,NaN
4,Dhaka,41923,1981,5,0,0,0,20,8,0,...,0,13,1,1,1,43,45.0,0,39,NaN


In [42]:
for col in ['D_29', 'D_30', 'D_31']:
    if col in df.columns:
        df[col] = df[col].fillna(0)
df.replace('***', np.nan, inplace=True)
df.head()

,Station_Name,Station_ID,Year,Month,D_01,D_02,D_03,D_04,D_05,D_06,...,D_23,D_24,D_25,D_26,D_27,D_28,D_29,D_30,D_31,Unnamed: 35
0,Dhaka,41923,1981,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0.0,6.0,9.0,0.0
1,Dhaka,41923,1981,2,2,0,0,0,0,0,...,0,0,0,0,0,0,0.0,0.0,0.0,0.0
2,Dhaka,41923,1981,3,0,0,0,0,0,0,...,10,0,13,0,8,0,2.0,0.0,41.0,0.0
3,Dhaka,41923,1981,4,2,0,0,4,0,0,...,0,0,0,0,0,0,3.0,0.0,0.0,0.0
4,Dhaka,41923,1981,5,0,0,0,20,8,0,...,0,13,1,1,1,43,45.0,0.0,39.0,0.0


In [43]:
# Drop the specified columns from the dataset
columns_to_drop = ['Station_Name', 'Station_ID','Unnamed: 35']
df_filtered = df.drop(columns=columns_to_drop)

# Display the first few rows of the filtered dataset
df_filtered.head()


,Year,Month,D_01,D_02,D_03,D_04,D_05,D_06,D_07,D_08,...,D_22,D_23,D_24,D_25,D_26,D_27,D_28,D_29,D_30,D_31
0,1981,1,0,0,0,0,0,0,2,0,...,0,0,0,0,0,0,0,0.0,6.0,9.0
1,1981,2,2,0,0,0,0,0,1,7,...,0,0,0,0,0,0,0,0.0,0.0,0.0
2,1981,3,0,0,0,0,0,0,0,0,...,0,10,0,13,0,8,0,2.0,0.0,41.0
3,1981,4,2,0,0,4,0,0,0,0,...,2,0,0,0,0,0,0,3.0,0.0,0.0
4,1981,5,0,0,0,20,8,0,0,0,...,0,0,13,1,1,1,43,45.0,0.0,39.0


In [44]:
# ধরুন df_filtered হল আপনার আগের ফিল্টার করা ডেটাসেট
time_series = []

for _, row in df_filtered.iterrows():
    year, month = int(row['Year']), int(row['Month'])
    for day in range(1, 32):
        col = f'D_{day:02d}'
        if col in row and not pd.isna(row[col]):
            try:
                date = pd.Timestamp(year=year, month=month, day=day)
                time_series.append((date, row[col]))
            except:
                continue

daily_df = pd.DataFrame(time_series, columns=['Date', 'Rainfall'])
daily_df.set_index('Date', inplace=True)
daily_df = daily_df.sort_index()

In [45]:
for lag in range(1, 8):  # গত 7 দিনের তাপমাত্রা
    daily_df[f'lag_{lag}'] = daily_df['Rainfall'].shift(lag)

daily_df.dropna(inplace=True)


In [46]:
from sklearn.model_selection import train_test_split

X = daily_df[[f'lag_{i}' for i in range(1, 8)]]
y = daily_df['Rainfall']

X_train, X_test, y_train, y_test = train_test_split(X, y, shuffle=False, test_size=365*5)  # শেষ 5 বছর টেস্ট


In [47]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

model = RandomForestRegressor(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

y_pred = model.predict(X_test)
print("MSE:", mean_squared_error(y_test, y_pred))


MSE: 178.5158445424617


In [48]:
from datetime import timedelta
import numpy as np

future_days = 365 * 8
last_known = daily_df.iloc[-7:]['Rainfall'].values.tolist()

future_preds = []
for _ in range(future_days):
    x_input = np.array(last_known[-7:]).reshape(1, -1)
    pred = model.predict(x_input)[0]
    future_preds.append(pred)
    last_known.append(pred)

future_dates = pd.date_range(start=daily_df.index[-1] + timedelta(days=1), periods=future_days)
future_df = pd.DataFrame({'Date': future_dates, 'Forecasted_Rainfall': future_preds})


Streaming output truncated to the last 5000 lines.
/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted 

In [50]:
future_df.to_csv('8_years_forecast_Rainfall_temp.csv')


In [11]:
# STEP 1: ফোরকাস্ট শুরু করব ২০২০-০১-০১ থেকে ৫ বছর
from datetime import timedelta
import numpy as np

# Ensure start date is in dataset
start_date = pd.Timestamp("2020-01-01")
start_idx = daily_df.index.get_loc(start_date)

# আগের ৭ দিনের ডেটা নিবো প্রেডিকশনের জন্য
last_known = daily_df.iloc[start_idx - 7:start_idx]['MaxTemp'].values.tolist()

future_days = 365 * 5  # 5 বছর

future_preds = []
for _ in range(future_days):
    x_input = np.array(last_known[-7:]).reshape(1, -1)
    pred = model.predict(x_input)[0]
    future_preds.append(pred)
    last_known.append(pred)

# STEP 2: তারিখের তালিকা বানাবো
future_dates = pd.date_range(start=start_date, periods=future_days)
future_df = pd.DataFrame({'Date': future_dates, 'Forecasted_MaxTemp': future_preds})
future_df.set_index('Date', inplace=True)


/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(
/usr/local/lib/

In [12]:
# CSV ফাইলে সংরক্ষণ
future_df.to_csv('forecast_2020_2024.csv')